In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
ctf = [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()]

# Load dataset
train_loader = FromPublic('cifar10', 'train', batch_size=512, num_workers=4, custom_tf=ctf)
test_loader = FromPublic('cifar10', 'test', batch_size=512, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
# Model
model = SENet18(input_dim=3)

# Learning
runner = Runner(model, train_loader, test_loader, optimizer='adam', loss_fn='focal')

In [23]:
for _ in range(3): runner.fit_onecycle(0.5, mixup_alpha=0.4)

KeyboardInterrupt: 

In [ ]:
runner.fit(5, 1e-3, mixup_alpha=0.4)

In [ ]:
runner.validate(metrics=[ConfusionMatrix(plot=True, classes=10)])

In [ ]:
runner.history(plot=True)

In [ ]:
runner.fit_cycle(5, 1e-4, mixup_alpha=0.4)

In [ ]:
runner.validate(metrics=[ConfusionMatrix(plot=True, classes=10)])

In [ ]:
runner.history(plot=True)